In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
train_df = pd.read_csv('/content/drive/MyDrive/DACON/Car_accident/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/DACON/Car_accident/test.csv')

In [ ]:
import numpy as np
from supervised import AutoML

In [ ]:
train_df['사고일시'] = pd.to_datetime(train_df['사고일시'])

In [ ]:
test_df['사고일시'] = pd.to_datetime(test_df['사고일시'])

# 군과 구를 나눠줍니다

In [ ]:
train_df['군'] = ''
train_df['구'] = ''

test_df['군'] = ''
test_df['구'] = ''

# 주소 분리 및 군과 구 열에 저장
for i in range(0, len(train_df)):
    address = train_df['시군구'][i]
    elements = address.split(" ")
    if len(elements) == 3:
        city = elements[0]
        county = elements[1]
        district = elements[2]
        train_df.at[i, '군'] = county
        train_df.at[i, '구'] = district
    else:
        print(f"잘못된 주소 형식입니다: {address}")

# 결과 출력
print(train_df)

                   ID                사고일시   요일 기상상태              시군구  \
0      ACCIDENT_00000 2019-01-01 00:00:00  화요일   맑음     대구광역시 중구 대신동   
1      ACCIDENT_00001 2019-01-01 00:00:00  화요일   흐림    대구광역시 달서구 감삼동   
2      ACCIDENT_00002 2019-01-01 01:00:00  화요일   맑음    대구광역시 수성구 두산동   
3      ACCIDENT_00003 2019-01-01 02:00:00  화요일   맑음     대구광역시 북구 복현동   
4      ACCIDENT_00004 2019-01-01 04:00:00  화요일   맑음     대구광역시 동구 신암동   
...               ...                 ...  ...  ...              ...   
39604  ACCIDENT_39604 2021-12-31 19:00:00  금요일   맑음  대구광역시 수성구 수성동3가   
39605  ACCIDENT_39605 2021-12-31 19:00:00  금요일   맑음    대구광역시 달서구 상인동   
39606  ACCIDENT_39606 2021-12-31 21:00:00  금요일   맑음    대구광역시 달서구 월성동   
39607  ACCIDENT_39607 2021-12-31 22:00:00  금요일   맑음     대구광역시 달서구 장동   
39608  ACCIDENT_39608 2021-12-31 23:00:00  금요일   맑음     대구광역시 서구 비산동   

                  도로형태 노면상태  사고유형 사고유형 - 세부분류       법규위반  ... 피해운전자 성별  \
0             단일로 - 기타   건조  차대사람  길가장자리구역통행중    안전운전불이행  ...

In [ ]:
for i in range(0, len(test_df)):
    address = test_df['시군구'][i]
    elements = address.split(" ")
    if len(elements) == 3:
        city = elements[0]
        county = elements[1]
        district = elements[2]
        test_df.at[i, '군'] = county
        test_df.at[i, '구'] = district
    else:
        print(f"잘못된 주소 형식입니다: {address}")

# 결과 출력
print(test_df)

                   ID                사고일시   요일 기상상태              시군구  \
0      ACCIDENT_39609 2022-01-01 01:00:00  토요일   맑음     대구광역시 수성구 상동   
1      ACCIDENT_39610 2022-01-01 01:00:00  토요일   맑음    대구광역시 수성구 지산동   
2      ACCIDENT_39611 2022-01-01 04:00:00  토요일   맑음  대구광역시 수성구 수성동2가   
3      ACCIDENT_39612 2022-01-01 04:00:00  토요일   맑음    대구광역시 수성구 신매동   
4      ACCIDENT_39613 2022-01-01 06:00:00  토요일   맑음    대구광역시 달서구 감삼동   
...               ...                 ...  ...  ...              ...   
10958  ACCIDENT_50567 2022-12-31 18:00:00  토요일   맑음     대구광역시 남구 대명동   
10959  ACCIDENT_50568 2022-12-31 18:00:00  토요일   맑음    대구광역시 수성구 시지동   
10960  ACCIDENT_50569 2022-12-31 20:00:00  토요일   맑음    대구광역시 수성구 연호동   
10961  ACCIDENT_50570 2022-12-31 20:00:00  토요일   맑음    대구광역시 수성구 범물동   
10962  ACCIDENT_50571 2022-12-31 21:00:00  토요일   맑음     대구광역시 동구 효목동   

              도로형태 노면상태  사고유형    군      구  
0       교차로 - 교차로안   건조  차대사람  수성구     상동  
1         단일로 - 기타   건조  차대사람  수성구    지산동  
2  

# 외부 데이터 사용

이 부분은 이희원님의 코드를 참고하였습니다.

In [ ]:
light_df = pd.read_csv('/content/drive/MyDrive/DACON/Car_accident/external_open/대구 보안등 정보.csv', encoding='cp949')[['설치개수', '소재지지번주소']]

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

light_df[['도시', '군', '구', '번지']] = light_df['소재지지번주소'].str.extract(location_pattern)
light_df = light_df.drop(columns=['소재지지번주소', '번지'])

light_df = light_df.groupby(['도시', '군', '구']).sum().reset_index()
light_df.reset_index(inplace=True, drop=True)

<ipython-input-9-544838195a4b>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.


In [ ]:
parking_df = pd.read_csv('/content/drive/MyDrive/DACON/Car_accident/external_open/대구 주차장 정보.csv', encoding='cp949')[['소재지지번주소', '급지구분']]
parking_df = pd.get_dummies(parking_df, columns=['급지구분'])

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

parking_df[['도시', '군', '구', '번지']] = parking_df['소재지지번주소'].str.extract(location_pattern)
parking_df = parking_df.drop(columns=['소재지지번주소', '번지'])

parking_df = parking_df.groupby(['도시', '군', '구']).sum().reset_index()
parking_df.reset_index(inplace=True, drop=True)

In [ ]:
child_area_df = pd.read_csv('/content/drive/MyDrive/DACON/Car_accident/external_open/대구 어린이 보호 구역 정보.csv', encoding='cp949').drop_duplicates()[['소재지지번주소']]
child_area_df['cnt'] = 1

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

child_area_df[['도시', '군', '구', '번지']] = child_area_df['소재지지번주소'].str.extract(location_pattern)
child_area_df = child_area_df.drop(columns=['소재지지번주소', '번지'])

child_area_df = child_area_df.groupby(['도시', '군', '구']).sum().reset_index()
child_area_df.reset_index(inplace=True, drop=True)

In [ ]:
train_df = pd.merge(train_df, light_df, how='left', on=['군', '구'])
train_df = pd.merge(train_df, child_area_df, how='left', on=['군', '구'])
train_df = pd.merge(train_df, parking_df, how='left', on=['군', '구'])

test_df = pd.merge(test_df, light_df, how='left', on=['군', '구'])
test_df = pd.merge(test_df, child_area_df, how='left', on=['군', '구'])
test_df = pd.merge(test_df, parking_df, how='left', on=['군', '구'])

In [ ]:
test_df['군'].unique()

array(['수성구', '달서구', '중구', '동구', '서구', '북구', '남구', '달성군'], dtype=object)

## One-hot-encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# train_df와 test_df의 기상상태 및 노면상태 열 선택
train_categorical_data = train_df[['군']]
test_categorical_data = test_df[['군']]

# OneHotEncoder 인스턴스 생성 및 fit_transform 수행
encoder = OneHotEncoder()
train_encoded = encoder.fit_transform(train_categorical_data)
test_encoded = encoder.transform(test_categorical_data)

# OneHotEncoder가 사용한 카테고리 목록을 가져와서 카테고리 이름을 열 이름으로 변환
feature_names = encoder.get_feature_names_out(['군'])

# 밀집 행렬로 변환 (선택 사항)
train_encoded_dense = train_encoded.toarray()
test_encoded_dense = test_encoded.toarray()

# 데이터프레임으로 변환 (선택 사항)
import pandas as pd

train_encoded_df = pd.DataFrame(train_encoded_dense, columns=feature_names, index=train_df.index)
test_encoded_df = pd.DataFrame(test_encoded_dense, columns=feature_names, index=test_df.index)

# 기존 열 제거
train_df = train_df.drop(['군', ], axis=1)
test_df = test_df.drop(['군'], axis=1)

# 인코딩된 열 추가
train_df = pd.concat([train_df, train_encoded_df], axis=1)
test_df = pd.concat([test_df, test_encoded_df], axis=1)


In [ ]:
train_df

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,급지구분_2,급지구분_3,군_남구,군_달서구,군_달성군,군_동구,군_북구,군_서구,군_수성구,군_중구
0,ACCIDENT_00000,2019-01-01 00:00:00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,ACCIDENT_00001,2019-01-01 00:00:00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ACCIDENT_00002,2019-01-01 01:00:00,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,ACCIDENT_00003,2019-01-01 02:00:00,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,9.0,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,ACCIDENT_00004,2019-01-01 04:00:00,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,ACCIDENT_39604,2021-12-31 19:00:00,금요일,맑음,대구광역시 수성구 수성동3가,교차로 - 교차로안,건조,차대차,측면충돌,신호위반,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
39605,ACCIDENT_39605,2021-12-31 19:00:00,금요일,맑음,대구광역시 달서구 상인동,단일로 - 기타,건조,차대차,측면충돌,안전거리미확보,...,0.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
39606,ACCIDENT_39606,2021-12-31 21:00:00,금요일,맑음,대구광역시 달서구 월성동,교차로 - 교차로안,건조,차대차,측면충돌,교차로운행방법위반,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
39607,ACCIDENT_39607,2021-12-31 22:00:00,금요일,맑음,대구광역시 달서구 장동,기타 - 기타,건조,차대차,추돌,안전운전불이행,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


# 시군구별 발생횟수 feature 추가

In [ ]:
accident_counts = train_df['시군구'].value_counts().reset_index()
accident_counts.columns = ['시군구', '사고횟수']

In [ ]:
# '시군구'별 사고 횟수를 train_df에 매핑하기 위해 '시군구' 컬럼을 기준으로 사고 횟수를 가져옵니다.
train_df['사고발생횟수'] = train_df['시군구'].map(accident_counts.set_index('시군구')['사고횟수'])


In [ ]:
# '시군구'별 사고 횟수를 train_df에 매핑하기 위해 '시군구' 컬럼을 기준으로 사고 횟수를 가져옵니다.
test_df['사고발생횟수'] = test_df['시군구'].map(accident_counts.set_index('시군구')['사고횟수'])


In [ ]:
train_df

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,급지구분_3,군_남구,군_달서구,군_달성군,군_동구,군_북구,군_서구,군_수성구,군_중구,사고발생횟수
0,ACCIDENT_00000,2019-01-01 00:00:00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,131
1,ACCIDENT_00001,2019-01-01 00:00:00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,678
2,ACCIDENT_00002,2019-01-01 01:00:00,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,604
3,ACCIDENT_00003,2019-01-01 02:00:00,화요일,맑음,대구광역시 북구 복현동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,426
4,ACCIDENT_00004,2019-01-01 04:00:00,화요일,맑음,대구광역시 동구 신암동,단일로 - 기타,건조,차대차,추돌,안전운전불이행,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39604,ACCIDENT_39604,2021-12-31 19:00:00,금요일,맑음,대구광역시 수성구 수성동3가,교차로 - 교차로안,건조,차대차,측면충돌,신호위반,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,104
39605,ACCIDENT_39605,2021-12-31 19:00:00,금요일,맑음,대구광역시 달서구 상인동,단일로 - 기타,건조,차대차,측면충돌,안전거리미확보,...,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1143
39606,ACCIDENT_39606,2021-12-31 21:00:00,금요일,맑음,대구광역시 달서구 월성동,교차로 - 교차로안,건조,차대차,측면충돌,교차로운행방법위반,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,618
39607,ACCIDENT_39607,2021-12-31 22:00:00,금요일,맑음,대구광역시 달서구 장동,기타 - 기타,건조,차대차,추돌,안전운전불이행,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,125


In [ ]:
# datetime 컬럼 처리
train_df['사고일시'] = pd.to_datetime(train_df['사고일시'])
test_df['사고일시'] = pd.to_datetime(test_df['사고일시'])

# datetime을 여러 파생 변수로 변환
for df in [train_df, test_df]:
    df['year'] = df['사고일시'].dt.year
    df['month'] = df['사고일시'].dt.month
    df['day'] = df['사고일시'].dt.day
    df['hour'] = df['사고일시'].dt.hour
    df['minute'] = df['사고일시'].dt.minute
    df['weekday'] = df['사고일시'].dt.weekday

In [ ]:
# holi_weekday = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
#                 '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
#                 '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
#                 '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
#                 '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01']

# 대체 휴일까지 포
holi_weekday = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-06', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
           '2020-01-01', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-03-01', '2020-04-15', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25', '2021-01-01',
           '2021-02-11', '2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-08-16', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-04', '2021-10-09', '2021-10-11', '2021-12-25',
           '2022-01-01', '2022-01-31', '2022-02-01', '2022-02-02', '2022-03-01', '2022-03-09', '2022-05-05', '2022-05-08', '2022-06-01', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2022-10-10', '2022-12-25']

In [ ]:
train_df['사고일시'] = pd.to_datetime(train_df['사고일시'])
train_df['day_of_week'] = train_df['사고일시'].dt.dayofweek
train_df['holiday'] = np.where((train_df.day_of_week >= 5) | (train_df.사고일시.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)

In [ ]:
test_df['사고일시'] = pd.to_datetime(test_df['사고일시'])
test_df['day_of_week'] = test_df['사고일시'].dt.dayofweek
test_df['holiday'] = np.where((test_df.day_of_week >= 5) | (test_df.사고일시.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)

In [ ]:
train_df = train_df.drop(columns=['사고유형 - 세부분류','법규위반','가해운전자 차종','가해운전자 성별','가해운전자 연령',
                               '가해운전자 상해정도','사망자수','사망자수','중상자수','피해운전자 차종','피해운전자 성별',
                      '피해운전자 연령','피해운전자 상해정도','경상자수','부상자수','사고유형 - 세부분류'])

# 계절 변수 추가

In [ ]:
def group_season(df):
    df.loc[(df['month'] == 3) | (df['month'] == 4) | (df['month'] == 5), 'season'] = '봄'
    df.loc[(df['month'] == 6) | (df['month'] == 7) | (df['month'] == 8), 'season'] = '여름'
    df.loc[(df['month'] == 9) | (df['month'] == 10) | (df['month'] == 11), 'season'] = '가을'
    df.loc[(df['month'] == 12) | (df['month'] == 1) | (df['month'] == 2), 'season'] = '겨울'
    return df['season']

# train_df['Sine_Time'] = np.sin(2 * np.pi * train_df['hour'] / 24)
# test_df['Sine_Time'] = np.sin(2 * np.pi * test_df['hour'] / 24)
train_df['Cosine_Time'] = np.cos(2 * np.pi * train_df['hour'] / 24)
test_df['Cosine_Time'] = np.cos(2 * np.pi * test_df['hour'] / 24)
train_df['season'] = group_season(train_df)
test_df['season'] = group_season(test_df)

# Target-Encoding

In [ ]:
# Target encoding
from category_encoders.target_encoder import TargetEncoder

categorical_features = ['ID','시군구','요일','도로형태','사고유형','구','기상상태', '노면상태','season']

for i in categorical_features:
    tr_encoder = TargetEncoder(cols=[i])
    train_df[i] = tr_encoder.fit_transform(train_df[i], train_df['ECLO'])
    test_df[i] = tr_encoder.transform(test_df[i])

In [ ]:
target = train_df['ECLO']

In [ ]:
train_df['ECLO'] = target

In [ ]:
train_df.corr()

<ipython-input-29-a7a80d20a44e>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.


,ID,요일,기상상태,시군구,도로형태,노면상태,사고유형,ECLO,구,설치개수,...,year,month,day,hour,minute,weekday,day_of_week,holiday,Cosine_Time,season
ID,1.000000,0.061771,0.014750,0.118496,0.086154,0.018617,0.137211,1.000000,0.117913,-0.021686,...,-0.031281,-0.006871,-0.012303,-0.017355,NaN,0.039407,0.039407,0.062102,0.037052,0.013063
요일,0.061771,1.000000,0.005726,-0.011276,0.009456,0.003387,0.010758,0.061771,-0.011102,-0.000137,...,-0.015829,-0.000869,-0.001398,-0.028288,NaN,0.637956,0.637956,0.853391,0.031918,0.010607
기상상태,0.014750,0.005726,1.000000,0.006730,0.005441,0.811933,-0.019851,0.014750,0.007334,0.002039,...,0.028432,-0.019630,-0.012132,0.025395,NaN,-0.007867,-0.007867,0.015571,0.099668,0.019916
시군구,0.118496,-0.011276,0.006730,1.000000,0.025979,-0.005277,0.093599,0.118496,0.995708,-0.188473,...,0.013896,0.011130,0.001173,-0.014139,NaN,-0.010625,-0.010625,-0.007478,-0.046177,0.007512
도로형태,0.086154,0.009456,0.005441,0.025979,1.000000,0.019300,0.238355,0.086154,0.026132,-0.019407,...,-0.008411,-0.002795,0.006277,-0.014810,NaN,0.008161,0.008161,0.008271,0.022999,-0.002123
노면상태,0.018617,0.003387,0.811933,-0.005277,0.019300,1.000000,-0.008089,0.018617,-0.004705,0.007760,...,0.018757,-0.023078,-0.017716,0.019702,NaN,-0.013314,-0.013314,0.010902,0.081766,0.016906
사고유형,0.137211,0.010758,-0.019851,0.093599,0.238355,-0.008089,1.000000,0.137211,0.092879,-0.029901,...,0.008337,0.000300,-0.000257,0.009275,NaN,0.009591,0.009591,0.009229,-0.042967,0.001176
ECLO,1.000000,0.061771,0.014750,0.118496,0.086154,0.018617,0.137211,1.000000,0.117913,-0.021686,...,-0.031281,-0.006871,-0.012303,-0.017355,NaN,0.039407,0.039407,0.062102,0.037052,0.013063
구,0.117913,-0.011102,0.007334,0.995708,0.026132,-0.004705,0.092879,0.117913,1.000000,-0.189628,...,0.014204,0.011202,0.001162,-0.013153,NaN,-0.011023,-0.011023,-0.007313,-0.045888,0.007619
설치개수,-0.021686,-0.000137,0.002039,-0.188473,-0.019407,0.007760,-0.029901,-0.021686,-0.189628,1.000000,...,-0.006312,0.007981,0.002052,0.001204,NaN,-0.001479,-0.001479,0.001967,-0.007323,0.009813


In [ ]:
#train_df = train_df.drop(columns=['ECLO'])

In [ ]:
#test_df = test_df.drop(columns=['사고일시'])

In [ ]:
train_df.columns

Index(['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', 'ECLO', '구',
       '도시_x', '설치개수', '도시_y', 'cnt', '도시', '급지구분_1', '급지구분_2', '급지구분_3',
       '군_남구', '군_달서구', '군_달성군', '군_동구', '군_북구', '군_서구', '군_수성구', '군_중구',
       '사고발생횟수', 'year', 'month', 'day', 'hour', 'minute', 'weekday',
       'day_of_week', 'holiday', 'Cosine_Time', 'season'],
      dtype='object')

## 비비드백님의 인사이트를 참고하였습니다

In [ ]:
test_df

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,구,도시_x,...,year,month,day,hour,minute,weekday,day_of_week,holiday,Cosine_Time,season
0,4.726704,2022-01-01 01:00:00,4.920811,4.712888,4.881657,5.006142,4.712878,3.817650,4.881657,대구광역시,...,2022,1,1,1,0,5,5,1,9.659258e-01,4.659111
1,4.726704,2022-01-01 01:00:00,4.920811,4.712888,4.563008,4.653990,4.712878,3.817650,4.563008,NaN,...,2022,1,1,1,0,5,5,1,9.659258e-01,4.659111
2,4.726704,2022-01-01 04:00:00,4.920811,4.712888,4.945578,5.006142,4.712878,4.944597,4.945578,NaN,...,2022,1,1,4,0,5,5,1,5.000000e-01,4.659111
3,4.726704,2022-01-01 04:00:00,4.920811,4.712888,4.438172,4.653990,4.712878,4.944597,4.438172,NaN,...,2022,1,1,4,0,5,5,1,5.000000e-01,4.659111
4,4.726704,2022-01-01 06:00:00,4.920811,4.712888,4.738938,5.006142,4.712878,4.944597,4.738938,대구광역시,...,2022,1,1,6,0,5,5,1,6.123234e-17,4.659111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,4.726704,2022-12-31 18:00:00,4.920811,4.712888,4.492281,6.555670,4.712878,4.944597,4.492281,대구광역시,...,2022,12,31,18,0,5,5,1,-1.836970e-16,4.659111
10959,4.726704,2022-12-31 18:00:00,4.920811,4.712888,4.928144,4.653990,4.712878,4.944597,4.928144,NaN,...,2022,12,31,18,0,5,5,1,-1.836970e-16,4.659111
10960,4.726704,2022-12-31 20:00:00,4.920811,4.712888,5.355036,4.653990,4.712878,4.944597,5.355036,NaN,...,2022,12,31,20,0,5,5,1,5.000000e-01,4.659111
10961,4.726704,2022-12-31 20:00:00,4.920811,4.712888,5.079498,4.863035,4.712878,4.944597,5.079498,NaN,...,2022,12,31,20,0,5,5,1,5.000000e-01,4.659111


In [ ]:
test_x = test_df.drop(columns=['ID', '도시_x','도시_y','도시', 'month','day','minute','day_of_week','사고일시']).copy()
train_x = train_df[test_x.columns].copy()
train_y = train_df['ECLO'].copy()

In [ ]:
test_x.columns

Index(['요일', '도로형태', '사고유형', '구', '설치개수', 'cnt', '급지구분_1', '급지구분_2', '급지구분_3',
       '군_남구', '군_달서구', '군_달성군', '군_동구', '군_북구', '군_서구', '군_수성구', '군_중구',
       '사고발생횟수', 'year', 'hour', 'weekday', 'holiday', 'Cosine_Time',
       'season'],
      dtype='object')

In [ ]:
test_x

,요일,도로형태,사고유형,구,설치개수,cnt,급지구분_1,급지구분_2,급지구분_3,군_남구,...,군_서구,군_수성구,군_중구,사고발생횟수,year,hour,weekday,holiday,Cosine_Time,season
0,4.920811,5.006142,3.817650,4.881657,700.0,5.0,NaN,NaN,NaN,0.0,...,0.0,1.0,0.0,338,2022,1,5,1,9.659258e-01,4.659111
1,4.920811,4.653990,3.817650,4.563008,NaN,10.0,0.0,0.0,2.0,0.0,...,0.0,1.0,0.0,492,2022,1,5,1,9.659258e-01,4.659111
2,4.920811,5.006142,4.944597,4.945578,NaN,1.0,NaN,NaN,NaN,0.0,...,0.0,1.0,0.0,147,2022,4,5,1,5.000000e-01,4.659111
3,4.920811,4.653990,4.944597,4.438172,NaN,7.0,0.0,2.0,1.0,0.0,...,0.0,1.0,0.0,372,2022,4,5,1,5.000000e-01,4.659111
4,4.920811,5.006142,4.944597,4.738938,932.0,NaN,0.0,1.0,3.0,0.0,...,0.0,0.0,0.0,678,2022,6,5,1,6.123234e-17,4.659111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10958,4.920811,6.555670,4.944597,4.492281,5377.0,26.0,20.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1749,2022,18,5,1,-1.836970e-16,4.659111
10959,4.920811,4.653990,4.944597,4.928144,NaN,5.0,NaN,NaN,NaN,0.0,...,0.0,1.0,0.0,167,2022,18,5,1,-1.836970e-16,4.659111
10960,4.920811,4.653990,4.944597,5.355036,NaN,NaN,NaN,NaN,NaN,0.0,...,0.0,1.0,0.0,107,2022,20,5,1,5.000000e-01,4.659111
10961,4.920811,4.863035,4.944597,5.079498,NaN,7.0,NaN,NaN,NaN,0.0,...,0.0,1.0,0.0,239,2022,20,5,1,5.000000e-01,4.659111


In [ ]:
# 데이터프레임의 모든 NaN 값을 0으로 대체
train_x.fillna(0, inplace=True)
test_x.fillna(0, inplace=True)

In [ ]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

### AUTOML

In [ ]:
import matplotlib
matplotlib.use('Agg')

In [ ]:
from supervised.automl import AutoML

automl = AutoML(mode="Compete",
                algorithms = ['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost'],
                n_jobs = -1,total_time_limit=14400, eval_metric="rmse", ml_task = "regression",
                )
#3.209155

In [ ]:
automl.fit(train_x, train_y)

AutoML directory: AutoML_1
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree rmse 3.217235 trained in 1.64 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 4 models
1_Default_LightGBM rmse 3.234101 trained in 22.31 seconds
There was an error during 2_Default_Xgboost training.
Please check AutoML_1/errors.md for details.
There was an error during 3_De

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to '

There was an error during 36_RandomForest_KMeansFeatures training.
Please check AutoML_1/errors.md for details.


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to '

There was an error during 37_RandomForest_KMeansFeatures training.
Please check AutoML_1/errors.md for details.


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to '

There was an error during 25_CatBoost_KMeansFeatures training.
Please check AutoML_1/errors.md for details.
* Step insert_random_feature will try to check up to 1 model
A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.
Problem during computing permutation importance. Skipping ...
A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.
Problem during computing permutation importance. Skipping ...
A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.
Problem during computing permutation importance. Skipping ...
A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.
Problem during computing permutation importance. Skipping ...
A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.
Problem during computing permutation importance. Skipping ...
A task

ImportError: ignored

In [ ]:
pred = automl.predict(test_x)

## 제출

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/DACON/Car_accident/sample_submission.csv')

In [ ]:
submission['ECLO'] = pred

In [ ]:
submission.loc[ submission['ECLO'] < 0.0, 'ECLO'] = 0.0

In [ ]:
min(submission['ECLO'])

2.7588669377862667

In [ ]:
submission.to_csv('/content/drive/MyDrive/DACON/Car_accident/submit.csv', index=False)

In [ ]:
submission

,ID,ECLO
0,ACCIDENT_39609,3.808590
1,ACCIDENT_39610,3.501276
2,ACCIDENT_39611,4.781229
3,ACCIDENT_39612,4.262799
4,ACCIDENT_39613,4.445028
...,...,...
10958,ACCIDENT_50567,4.961047
10959,ACCIDENT_50568,4.393577
10960,ACCIDENT_50569,4.784851
10961,ACCIDENT_50570,4.649927
